# **<span style="font-size:1em;">PrOMMiS LCA Integration: Environmental Life Cycle Assessment Using PrOMMiS Modeling Results</span>**

## Introduction

This document demonstrates the application of Life Cycle Assessment (LCA) methodology to the optimized results from the PrOMMiS model.  
This Jupyter Notebook presents an example LCA application to the West Kentucky No.13 Coal Refuse Flowsheet optimized in the PrOMMiS model.

**Useful notes, definitions, and links:**
- **PrOMMiS model:** Process Optimization and Modeling for Minerals Sustainability initiative, led by the U.S. Department of Energy (DOE), specifically under NETL (National Energy Technology Laboratory).
- **PrOMMiS public GitHub Repository:** [https://github.com/prommis/prommis/tree/main](https://github.com/prommis/prommis/tree/main)
- **UKy Flowsheet Code:** [https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py](https://github.com/prommis/prommis/blob/main/src/prommis/uky/uky_flowsheet.py)

<div style="text-align: center;">
    <img src="images\uky_flowsheet.png" width="1000"/>
</div>

## Step 1: Import the necessary tools

In [1]:
# import main libraries
import pandas as pd
import numpy as np
import olca_ipc
import olca_schema as olca
from netlolca.NetlOlca import NetlOlca
from typing import Union, List, Optional

from pyomo.environ import (units,value,)

import pubchempy as pcp

from pymatgen.core import Composition

from warnings import warn

# module to run the PrOMMiS model
import prommis.uky.uky_flowsheet as uky
# module to extract the data from the PrOMMiS model
from src import prommis_LCA_data
# module to run convert the PrOMMiS model data to LCA data
from src import prommis_LCA_conversions
# module to finalize the LCA data
from src import finalize_LCA_flows

# modules to connect to openLCA
from src.create_olca_process.olca_utils import (
    initialize_client, print_database_summary, generate_id, find_entity_by_name
)
from src.create_olca_process.dataframe_utils import (
    validate_dataframe_columns, load_dataframe_from_csv, get_amount_from_dataframe_row
)
from src.create_olca_process.search_utils import (
    get_existing_processes, search_processes_by_keywords
)
from src.create_olca_process.user_interface import (
    show_process_selection_menu, get_yes_no_input, get_user_search_choice, 
    show_unit_selection_in_flow_menu, show_flow_process_selection_menu
)
from src.create_olca_process.process_manager import create_process_from_dataframe_with_selection
from src.create_olca_process.flow_manager import find_compatible_flow_pattern, search_flows_by_name
from src.create_olca_process.unit_manager import (
    get_existing_units, get_common_unit_suggestions, search_units_by_name, 
    find_units_in_flow, find_unit_in_flow_by_name, create_unit_from_dataframe_row
)

✅ olca-ipc and olca-schema imported successfully


## Step 2: Run PrOMMiS Optimization Model for the UKy flowsheet and extract model results 

In [2]:
# Run PrOMMiS model 
m, results = uky.main()

# Extract LCA data
prommis_data = prommis_LCA_data.get_lca_df(m, )
prommis_data.to_csv("lca_df.csv")

2025-08-05 12:24:09,873 - INFO - Starting Sequential Decomposition
2025-08-05 12:24:09,876 - INFO - Starting first pass run of network


Initialization Order
fs.leach_solid_feed
fs.leach
fs.sl_sep1
fs.leach_mixer
fs.sc_circuit_purge
fs.precip_sx_mixer
2025-08-05 12:24:09 [INFO] idaes.prommis.uky.uky_flowsheet: Initializing fs.leach_solid_feed


2025-08-05 12:24:09,877 - INFO - Initializing fs.leach_solid_feed


2025-08-05 12:24:09 [INFO] idaes.prommis.uky.uky_flowsheet: Initializing fs.leach_liquid_feed


2025-08-05 12:24:09,887 - INFO - Initializing fs.leach_liquid_feed


...

2025-08-05 12:24:16 [INFO] idaes.prommis.uky.uky_flowsheet: Initializing fs.precip_purge


2025-08-05 12:24:16,245 - INFO - Initializing fs.precip_purge
2025-08-05 12:24:16,264 - INFO - Finished Sequential Decomposition in 6.39 seconds



Unit : fs.roaster                                                          Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance


    Expressions: 

    Key                      : Value      : Units
    Product Al Mass Fraction : 3.2488e-05 : dimensionless
    Product Ca Mass Fraction :  0.0011611 : dimensionless
    Product Ce Mass Fraction :    0.43801 : dimensionless
    Product Dy Mass Fraction :   0.011776 : dimensionless
    Product Fe Mass Fraction : 8.5623e-06 : dimensionless
    Product Gd Mass Fraction :   0.049882 : dimensionless
    Product La Mass Fraction :    0.13882 : dimensionless
    Product Nd Mass Fraction :    0.24543 : dimensionless
    Product Pr Mass Fraction :   0.054681 : dimensionless
    Product Sc Mass Fraction : 0.00032940 : dimensionless
    Product Sm Mass Fraction :   0.025317 : dimensionless
     Product Y Mass Fraction :   0.034553 : dimensionless

----------------------------------------

2025-08-05 12:24:17,580 - WARNING - model contains export suffix 'scaling_factor' that contains 16 component keys that are not exported as part of the NL file.  Skipping.
2025-08-05 12:24:17,776 - WARNING - model contains export suffix 'scaling_factor' that contains 16 component keys that are not exported as part of the NL file.  Skipping.



costing
------------------------------------------------------------------------------------
                                                                             Value     
    Plant Cost Units                                                      MUSD_UKy_2019
    Total Plant Cost                                                            0.97937
    Total Bare Erected Cost                                                     0.32975
    Total Installation Cost                                                     0.64961
    Total Other Plant Costs                                                  6.2342e-06
    Summation of Ancillary Installation Costs                                   0.19126
    Total Ancillary Piping, Materials and Labor Installation Cost              0.065951
    Total Ancillary Electrical, Materials and Labor Installation Cost          0.065951
    Total Ancillary Instrumentation Installation Cost                          0.026380
    Total Ancillary Plant 

## Step 3: Review PrOMMiS Results

In [3]:
prommis_data.head()

,Flow,Source,In/Out,Category,Value 1,Unit 1,Value 2,Unit 2
0,Inerts,Solid Feed,In,Solid Input,22.68,kg/hr,0.6952,mass fraction
1,Scandium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000028,mass fraction
2,Yttrium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000033,mass fraction
3,Lanthanum Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000068,mass fraction
4,Cerium Oxide,Solid Feed,In,Solid Input,22.68,kg/hr,0.000156,mass fraction


## Step 4: Organize, Categorize, and Covert PrOMMiS Results to LCA Relevant Flows

In [ ]:
# Hours is the time period for the conversion
# mol_to_kg is a boolean that indicates whether to convert moles to kg
# water_unit is the unit of water (m3 or L recommended, kg is also an option)
df = prommis_LCA_conversions.convert_flows_to_lca_units(prommis_data, hours=1, mol_to_kg=True, water_unit='m3')
df.to_csv('lca_df_converted.csv', index=False)

# Finalize the LCA data
df = pd.read_csv('lca_df_converted.csv')
df = finalize_LCA_flows.main()
df.to_csv('lca_df_finalized.csv', index=False)

# This will create a new datafrane and csv file that contains the initial prommis_data df with two additional columns:
# 'LCA Amount' and 'LCA Unit'

We made it
l
We made it
l
We made it
l
We made it
l


2025-08-05 12:24:19,452 - WARNING - Dynamically registering the following numeric type:
    pymatgen.core.units.FloatWithUnit
Dynamic registration is supported for convenience, but there are known
limitations to this approach.  We recommend explicitly registering
numeric types using RegisterNumericType() or RegisterIntegerType().


Applied functional unit conversion with scaling factor: 0.0017916145175811875
Reference flow: 99.85% REO Product from Roaster Product
Summary:
  total_flows: 17
  input_flows: 10
  output_flows: 7
  reference_products: 1
  unique_flow_types: 4
  total_lca_amount: 300253.12066288554

Flow Type Breakdown:
  Technosphere flows: 10
  emission/air: 4
  Waste flows: 2
  Resource/water: 1


## Step 6: Normalize Flows to the Selected Functional Unit and Review Final LCA Flows

In [5]:
# The below code will normalize the converted prommis data to a functional unit
# Note: This code is developed for the UKy flowsheet and the functional unit is automatically set to 1 kg of REO (combination of all REEs)
# TODO: This code should be developed for the other flowsheets. To achieve this, the user should be able to specify the desired functional unit
df = pd.read_csv('lca_df_converted.csv')

# Run the merge_flows function for the feed
REO_list = [
    "Yttrium Oxide",
    "Lanthanum Oxide",
    "Cerium Oxide",
    "Praseodymium Oxide",
    "Neodymium Oxide",
    "Samarium Oxide",
    "Gadolinium Oxide",
    "Dysprosium Oxide",
]
df = finalize_LCA_flows.merge_flows(df, merge_source='Solid Feed', new_flow_name='374 ppm REO Feed', value_2_merge=REO_list)
# This 374 ppm value is directly calculated from the flowsheet. The original study actually used 357 ppm as the feed concentration.

# Run the merge_flows function for the product
df = finalize_LCA_flows.merge_flows(df, merge_source='Roaster Product', new_flow_name='99.85% REO Product')

# Run the merge_flows function for the liquid waste flows
df = finalize_LCA_flows.merge_flows(df, merge_source='Wastewater', new_flow_name='Wastewater', merge_column='Category') 
# Note: some of these streams are organic waste, but they're treated as wastewater

# Run the merge_flows function for the solid waste flows
df = finalize_LCA_flows.merge_flows(df, merge_source='Solid Waste', new_flow_name='Solid Waste', merge_column='Category') 

# Run the finalize_df function
try:
    finalized_df = finalize_LCA_flows.finalize_df(
        df=df, 
        reference_flow='99.85% REO Product', 
        reference_source='Roaster Product'
    )
    
    # Get summary
    summary = finalize_LCA_flows.get_finalize_summary(finalized_df)
    print("Summary:")
    for key, value in summary.items():
        if key != 'flow_type_breakdown':
            print(f"  {key}: {value}")
    
    print("\nFlow Type Breakdown:")
    for flow_type, count in summary['flow_type_breakdown'].items():
        print(f"  {flow_type}: {count}")
        
except Exception as e:
    print(f"Error during finalization: {e}")

finalized_df.to_csv('lca_df_finalized.csv', index=False)

Applied functional unit conversion with scaling factor: 0.0017916145175811875
Reference flow: 99.85% REO Product from Roaster Product
Summary:
  total_flows: 17
  input_flows: 10
  output_flows: 7
  reference_products: 1
  unique_flow_types: 4
  total_lca_amount: 300253.12066288554

Flow Type Breakdown:
  Technosphere flows: 10
  emission/air: 4
  Waste flows: 2
  Resource/water: 1


## Step 7: Connect to openLCA

### Notes on modeling this process using openLCA

Before running the following code:
- Open the software openLCA
- Open an existing database or create a new one
- Import libraries with existing processes
- Use the IPC server from the Developer Tools
- Run the server on port 8080 

In [6]:
print("\n🔌 STEP 1: Connecting to openLCA...")
client = initialize_client()

2025-08-05 12:24:20,619 - INFO - ✅ Connection test successful. Found 1741 processes.



🔌 STEP 1: Connecting to openLCA...
✅ Connected to openLCA


#### Define dataframe to be used

The dataframe should have the following columns: 

| Flow_Name | LCA_Unit | LCA_Amount | Is_Input | Reference_Product | Flow_Type | Description |

In [7]:
# Save the LCA data produced in previous steps in a dataframe called df
df = pd.read_csv('lca_df_finalized.csv')


#### Enter Process Information
##### Unit Process Name

In [8]:
process_name = "Test Process 1"  # Replace with the actual process name you want to use

#### Unit Process Description 

In [9]:
process_description = " Example Process Description"  # Replace with the actual process description you want to use

#### Enter Unit Process Flows

In [10]:
process = create_process_from_dataframe_with_selection(
    client=client,
    df=df,
    process_name=process_name,
    process_description=process_description
)



🔍 Processing 17 flows for process: Test Process 1

📋 Row 1: 374 ppm REO Feed
   Unit: kg, Amount: 12660.392829048247
   Type: Input

🔍 Search Options for Flow: 374 ppm REO Feed
💡 Choose search strategy:
   1. Search in PROCESSES (e.g., 'Coal power plant', 'Steel production')
   2. Search in FLOWS (e.g., 'CO2', 'Water', 'Electricity')
   3. Skip this flow

💡 Examples:
   - For 'Coal': Choose PROCESSES (search for coal power plants)
   - For 'CO2': Choose FLOWS (search for CO2 emissions flows)
   - For 'Water': Choose FLOWS (search for water flows)
⏭️  Skipping flow: 374 ppm REO Feed

📋 Row 2: 99.85% REO Product
   Unit: kg, Amount: 1.0
   Type: Output

🔍 Search Options for Flow: 99.85% REO Product
💡 Choose search strategy:
   1. Search in PROCESSES (e.g., 'Coal power plant', 'Steel production')
   2. Search in FLOWS (e.g., 'CO2', 'Water', 'Electricity')
   3. Skip this flow

💡 Examples:
   - For 'Coal': Choose PROCESSES (search for coal power plants)
   - For 'CO2': Choose FLOWS (searc

2025-08-05 12:26:07,878 - INFO - ✅ Successfully created process: Test Process 1 (ID: 7b22cd01-3db1-4db2-8143-19325ceb6037)


   ✅ Selected unit: l
   ✅ Exchange uses flow property: Mass
✅ Added exchange: Water, fresh - 130318.21170729032 l (Input)

💾 Attempting to save process to database...
   Process: Test Process 1
   ID: 7b22cd01-3db1-4db2-8143-19325ceb6037
   Exchanges: 2
✅ Process saved successfully to openLCA database!


## Step 8: Create Process in openLCA from the final dataframe produced in Step 6 